# Table of Contents
 <p><div class="lev1 toc-item"><a href="#load-necessary-information" data-toc-modified-id="load-necessary-information-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>load necessary information</a></div><div class="lev1 toc-item"><a href="#let's-look-at-an-example" data-toc-modified-id="let's-look-at-an-example-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>let's look at an example</a></div><div class="lev2 toc-item"><a href="#compounds" data-toc-modified-id="compounds-21"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>compounds</a></div><div class="lev2 toc-item"><a href="#forward-reaction" data-toc-modified-id="forward-reaction-22"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>forward reaction</a></div><div class="lev2 toc-item"><a href="#reverse-reaction" data-toc-modified-id="reverse-reaction-23"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>reverse reaction</a></div><div class="lev2 toc-item"><a href="#check-if-two-rates-are-equal" data-toc-modified-id="check-if-two-rates-are-equal-24"><span class="toc-item-num">2.4&nbsp;&nbsp;</span>check if two rates are equal</a></div><div class="lev2 toc-item"><a href="#causes-of-the-difference" data-toc-modified-id="causes-of-the-difference-25"><span class="toc-item-num">2.5&nbsp;&nbsp;</span>causes of the difference</a></div>

This notebook looks at how RMG finds degeneracy for double H_abstraction reactions where the products have identical reactants. It seeks to answer the question: Is degeneracy properly calculated when both products are the same. 

# load necessary information

In [1]:
import rmgpy.molecule
from rmgpy.molecule.symmetry import calculateCyclicSymmetryNumber
from rmgpy.molecule.molecule import Atom, Bond, Molecule
from rmgpy.species import Species
from rmgpy.data.kinetics.family import TemplateReaction
from rmgpy.reaction import Reaction
from rmgpy.data.base import ForbiddenStructures

In [2]:
from rmgpy.data.rmg import RMGDatabase
from rmgpy import settings
database = RMGDatabase()
database.load(
    path = settings['database.directory'], 
    thermoLibraries = ['primaryThermoLibrary'], # can add others if necessary
    kineticsFamilies=['H_Abstraction'],
    kineticsDepositories = 'all'
)
for family in database.kinetics.families.values():
    family.addKineticsRulesFromTrainingSet(thermoDatabase=database.thermo)
    family.fillKineticsRulesByAveragingUp(verbose=True)


In [3]:
# remove all forbidden structures
family = database.kinetics.families['H_Abstraction']
family.forbidden = ForbiddenStructures()

# let's look at an example

## compounds

In [4]:
ethyl = Molecule().fromSMILES('[CH2]C')
diethyl = Molecule().fromSMILES('[CH2][CH2]')
ethane = Molecule().fromSMILES('CC')

## forward reaction
This reaction has involves three species, the first one is the product twice

In [5]:
reactions = family.generateReactions(reactants=[diethyl,ethanle])
# remove products that don't match
for rxn in reactions:
    if any([ethyl.isIsomorphic(product) for product in rxn.products]):
        desiredReaction = rxn
        print rxn

NameError: name 'ethanle' is not defined

In [ ]:
desiredReaction

In [ ]:
template = family.retrieveTemplate(desiredReaction.template)
desiredReaction.kinetics = family.getKineticsForTemplate(template, degeneracy = desiredReaction.degeneracy)[0]
#convert ArrheniusEP to Arrhenius
desiredReaction.kinetics = desiredReaction.kinetics.toArrhenius(desiredReaction.getEnthalpyOfReaction(298))
reverse_kinetics = desiredReaction.generateReverseRateCoefficient()

In [ ]:
forward = desiredReaction

## reverse reaction
This reaction has involves three species, the first one reacts with itself in an identical way

In [ ]:
reactions = family.generateReactions(reactants=[ethyl,ethyl.copy(deep=True)])
for rxn in reactions:
    if any([diethyl.isIsomorphic(product) for product in rxn.products]):
        if any([ethane.isIsomorphic(product) for product in rxn.products]):
            desiredReaction = rxn
            print rxn

In [ ]:
desiredReaction

In [ ]:
template = family.retrieveTemplate(desiredReaction.template)
desiredReaction.kinetics = family.getKineticsForTemplate(template, degeneracy = desiredReaction.degeneracy)[0]
#convert ArrheniusEP to Arrhenius
desiredReaction.kinetics = desiredReaction.kinetics.toArrhenius(desiredReaction.getEnthalpyOfReaction(298))
forward_kinetics = desiredReaction.generateReverseRateCoefficient()

In [ ]:
reverse = desiredReaction

## check if two rates are equal

In [ ]:
abs(reverse_kinetics.getRateCoefficient(298) - reverse.kinetics.getRateCoefficient(298)) < 1e-7

In [ ]:
abs(forward_kinetics.getRateCoefficient(298) - forward.kinetics.getRateCoefficient(298)) < 1e-7

**the rates are NOT equal. They are off by a factor of 4** 

In [ ]:
forward.kinetics.getRateCoefficient(298)

In [ ]:
reverse.kinetics.getRateCoefficient(298)

## causes of the difference

In [ ]:
forward.degeneracy

In [ ]:
reverse.degeneracy

In [ ]:
forward.template

In [ ]:
reverse.template